# Create a new Java database and search model class

Specify the details of your new Java class below. 

## Rebuild the project OpenAPI spec, database scripts, and more

A running web application is made up of much more than Java code. 

- A data-driven website is also made up of CSS Stylesheets and Java Script. Each model has it's own set of specific JavaScript specific to it's fields and APIs. 

- An OpenAPI Spec is generated to map out all the APIs and pages in the site for system integrators to consume or update data in the site. 

- A database schema is also generated for storing all model data with specific field values, tables, and foreign key relations. 

Run the command below to generate all of the code listed above. 

Run the `index.sh` shell script of the `computate` project on  the  `smartvillage-platform` project to index all of the `smartvillage-platform` Java classes into the Apache Solr search engine. 

In [ ]:
eval $(./vars.py)
(cd $SITE_SRC && env \
  RUN_ARTICLE_GENERATOR=true \
  RUN_OPENAPI3_GENERATOR=true \
  RUN_PROJECT_GENERATOR=true \
  RUN_SQL_GENERATOR=true \
  VARS_PATH=$VARS_PATH \
  mvn exec:java -Dexec.mainClass=$SITE_MAIN_VERTICLE_PACKAGE.MainVerticle)
echo DONE

## Run SQL create scripts with new PostgreSQL tables
When we create new Smart Data Models that are persisted in the PostgreSQL database, we need to make sure that our database has all the tables and fields required to persist the data. Run the commands below to rsync the `db-create.sql` script to the PostgreSQL pod. It will connect again and apply all of the database schema changes to the database. 

In [ ]:
eval $(./vars.py)
oc exec pod/postgresql-0 -- mkdir -p /bitnami/postgresql/sql/
oc rsync $SITE_SRC/src/main/resources/sql/ \
  pod/postgresql-0:/bitnami/postgresql/sql/
oc exec pod/postgresql-0 -- \
  env PGPASSWORD=$(oc get secret/postgres-pguser-$DATABASE_USERNAME -o jsonpath={.data.password} | base64 -d) \
  psql -U $DATABASE_USERNAME -d $DATABASE_DATABASE -f /bitnami/postgresql/sql/db-create.sql
echo DONE

### Update your project MainVerticle.java Vert.x Verticle

In [ ]:
eval $(./vars.py)
ansible-playbook -e @$VARS_PATH playbooks/create-file.yaml \
  -e FILE_PATH="{{ SITE_SRC }}/src/main/java/{{ SITE_MAIN_VERTICLE_PACKAGE | replace('.', '/') }}/MainVerticle.java" \
  -e FILE_TEMPLATE=MainVerticle.java
echo DONE

### Update your project WorkerVerticle.java Vert.x Verticle

In [ ]:
eval $(./vars.py)
ansible-playbook -e @$VARS_PATH playbooks/create-file.yaml \
  -e FILE_PATH="{{ SITE_SRC }}/src/main/java/{{ SITE_MAIN_VERTICLE_PACKAGE | replace('.', '/') }}/WorkerVerticle.java" \
  -e FILE_TEMPLATE=WorkerVerticle.java
echo DONE

### Create your project PageLayout.htm Jinja2 HTML template

In [ ]:
eval $(./vars.py)
ansible-playbook -e @$VARS_PATH playbooks/create-file.yaml \
  -e FILE_PATH="{{ SITE_SRC }}/src/main/resources/templates/en-us/PageLayout.htm" \
  -e FILE_TEMPLATE=PageLayout.htm
echo DONE

## Create a Keycloak auth resource for your model


Create a Keycloak auth Resource for your new model in the site. 

In [ ]:
eval $(./vars.py)

ACCESS_TOKEN=$(curl -k -s -X POST -d "username=admin" -d "grant_type=password" -d "client_id=admin-cli" \
  -d "password=$(oc -n $AUTH_NAMESPACE get secret/keycloak-initial-admin -o jsonpath={.data.admin-password} | base64 -d)" \
  $AUTH_URL/realms/master/protocol/openid-connect/token | jq -r ".access_token")

curl -k -i -X POST -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
  "$AUTH_URL/admin/realms/$AUTH_REALM/clients/$AUTH_CLIENT/authz/resource-server/resource" \
  --data '{"name":"'"$CLASS_SIMPLE_NAME"'","displayName":"'"$CLASS_SIMPLE_NAME"'","scopes":["POST","PATCH","GET","'"$AUTH_SCOPE_ADMIN"'","'"$AUTH_SCOPE_SUPER_ADMIN"'"]}'

echo DONE

## Grant POST, PATCH, GET, SiteAdmin permissions for SiteAdmins to your new model


Create a Keycloak auth Permission for SiteAdmins to use the POST, PATCH, GET and SiteAdmin scopes of your model. 

In [ ]:
eval $(./vars.py)

ACCESS_TOKEN=$(curl -k -s -X POST -d "username=admin" -d "grant_type=password" -d "client_id=admin-cli" \
  -d "password=$(oc -n $AUTH_NAMESPACE get secret/keycloak-initial-admin -o jsonpath={.data.admin-password} | base64 -d)" \
  $AUTH_URL/realms/master/protocol/openid-connect/token | jq -r ".access_token")

curl -k -i -X POST -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
  "$AUTH_URL/admin/realms/$AUTH_REALM/clients/$AUTH_CLIENT/authz/resource-server/permission/scope" \
  --data '{"name":"group-'"$AUTH_SCOPE_ADMIN"'-resource-'"$CLASS_SIMPLE_NAME"'"
      ,"decisionStrategy":"AFFIRMATIVE"
      ,"description":"'"$AUTH_SCOPE_ADMIN"' group access to all '"$CLASS_SIMPLE_NAME"'"
      ,"resources":["'"$CLASS_SIMPLE_NAME"'"]
      ,"policies":["'"$AUTH_REALM"'-group-'"$AUTH_SCOPE_ADMIN"'"]
      ,"scopes":["'"$AUTH_REALM"'-POST","'"$AUTH_REALM"'-PATCH","'"$AUTH_REALM"'-GET","'"$AUTH_REALM"'-'"$AUTH_SCOPE_ADMIN"'"]
      }'

echo DONE

## Grant POST, PATCH, GET, SiteAdmin, SuperAdmin permissions for SuperAdmins to your new model


Create a Keycloak auth Permission for SuperAdmins to use the POST, PATCH, GET, SiteAdmin, and SuperAdmin scopes of your model. 

In [ ]:
eval $(./vars.py)

ACCESS_TOKEN=$(curl -k -s -X POST -d "username=admin" -d "grant_type=password" -d "client_id=admin-cli" \
  -d "password=$(oc -n $AUTH_NAMESPACE get secret/keycloak-initial-admin -o jsonpath={.data.admin-password} | base64 -d)" \
  $AUTH_URL/realms/master/protocol/openid-connect/token | jq -r ".access_token")

curl -k -i -X POST -H "Content-Type: application/json" -H "Authorization: Bearer $ACCESS_TOKEN" \
  "$AUTH_URL/admin/realms/$AUTH_REALM/clients/$AUTH_CLIENT/authz/resource-server/permission/scope" \
  --data '{"name":"group-'"$AUTH_SCOPE_SUPER_ADMIN"'-resource-'"$CLASS_SIMPLE_NAME"'"
      ,"decisionStrategy":"AFFIRMATIVE"
      ,"description":"'"$AUTH_SCOPE_SUPER_ADMIN"' group access to all '"$CLASS_SIMPLE_NAME"'"
      ,"resources":["'"$CLASS_SIMPLE_NAME"'"]
      ,"policies":["'"$AUTH_REALM"'-group-'"$AUTH_SCOPE_SUPER_ADMIN"'"]
      ,"scopes":["'"$AUTH_REALM"'-POST","'"$AUTH_REALM"'-PATCH","'"$AUTH_REALM"'-GET","'"$AUTH_REALM"'-'"$AUTH_SCOPE_ADMIN"'","'"$AUTH_REALM"'-'"$AUTH_SCOPE_SUPER_ADMIN"'"]
      }'

echo DONE

## Compile your Java project

In [ ]:
eval $(./vars.py)
(cd $SITE_SRC && mvn clean install)
echo DONE